In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install pymorphy2[fast]
!pip install navec

In [ ]:
from transformers import BertTokenizer
from transformers import AdamW, BertForMaskedLM
import torch
import numpy as np
import pymorphy2
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from navec import Navec
from typing import List
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import KeyedVectors
import scipy.spatial.distance as cos_dist
from tqdm import tqdm

random.seed(44)
morph = pymorphy2.MorphAnalyzer()
nltk.download('punkt')
navec = Navec.load(save_path + '/navec_hudlit_v1_12B_500K_300d_100q.tar')
save_path = '/content/drive/MyDrive'
alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

Using cpu


In [ ]:
categories = ("животные", "музыка", "спорт", "литература")
categories_eng = ("animals", "music", "sport", "literature")

with open(save_path + '/units.txt', 'r') as f:
  words = f.read()
  units = set(words.split('\n'))
with open(save_path + '/keywords_tasks.txt', 'r') as f:
  words = f.read()
  keywords_tasks = set(words.split('\n'))

all_nouns_actors = set()
nouns_actors = []
for ind, cat in enumerate(categories_eng):
  with open(save_path + f'/keywords/actors/true_keywords_nouns_actors_{cat}.txt', 'r') as f:
    words = f.read()
  nouns_actors.append([])
  for word in words.split('\n')[:-1]:
    if not word in units:
      nouns_actors[ind].append(word)
  all_nouns_actors |= set(nouns_actors[ind])

all_nouns = set()
nouns = []
for ind, cat in enumerate(categories_eng):
  with open(save_path + f'/keywords/nouns/true_keywords_nouns_{cat}.txt', 'r') as f:
    words = f.read()
  nouns.append([])
  for word in words.split('\n')[:-1]:
    if (not word in units) and (not word in keywords_tasks):
      nouns[ind].append(word)
  all_nouns |= set(nouns[ind])

In [ ]:
class MaskCreator():

  def __init__(self):
    self.tokenizer = nltk.WordPunctTokenizer()
    self.bigram_mod = gensim.models.Phrases.load(save_path + '/bigram_model.pkl')

  def make_bigrams(self, doc):
      return self.bigram_mod[doc]

  def mask(self, text, category=4) -> (str, List[str]):
    masks_dict = []
    tokens = self.tokenizer.tokenize(text.lower())
    tokens_normal = [morph.parse(w)[0].normal_form for w in tokens]
    tokens_bigrammed = self.make_bigrams(tokens_normal)
    
    if len(tokens_bigrammed) < len(tokens):
      ind_go = 0
      for i in range(len(tokens_bigrammed)):
        if tokens_normal[ind_go] != tokens_bigrammed[i]:
          tokens = tokens[:ind_go] + [tokens_bigrammed[i]] + tokens[ind_go+2:]
          ind_go += 2
        else:
          ind_go += 1

    if category == 4:
      now_keywords = all_nouns
    else:
      now_keywords = nouns[category]

    prev_words = []
    for ind, token in enumerate(tokens):
      word = morph.parse(token.lower())[0].normal_form
      if word in now_keywords:
        if word not in masks_dict:
          masks_dict.append(word)
        prev_words.append(tokens[ind])
        tokens[ind] = 'mask' + str(masks_dict.index(word, 0))
    text = nltk.tokenize.treebank.TreebankWordDetokenizer().detokenize(tokens)
    return text, masks_dict, prev_words

In [ ]:
all_keywords = [[] for i in range(4)]
already_in_keywords = set()
for word in navec.vocab.word_ids.keys():
  word_normal = morph.parse(word)[0].normal_form
  if (not word[0] in alphabet) or morph.parse(word)[0].tag.POS != 'NOUN' or word_normal in already_in_keywords:
    continue
  to_cats = []
  for cat in categories:
    to_cats.append(cos_dist.cosine(navec[word], navec[cat]))
  ind_min = np.argmin(to_cats)
  all_keywords[ind_min].append(word_normal)
  already_in_keywords.add(word_normal)

In [ ]:
#sentence = '4.30. У слона пульс 20 ударов в минуту, а у паука на 40 ударов в минуту больше. Какой пульс у паука?'
#sentence = '   1.1. Летом в спортивный лагерь ходили 50 детей, из них 9 девочек. Сколько мальчиков ходили в спортивный лагерь?'
#sentence = '   1.4. Во время летних соревнований по плаванию ребята посетили бассейн. Длина плавательной дорожки в бассейне 25 м. После того как первый участник соревнований проплыл часть дорожки, ему осталось проплыть 10 м. Сколько метров уже проплыл участник соревнований?'
#sentence = 'Используя данные круговой диаграммы, реши задачу. На диаграмме представлены данные о турпоходе группы. Сколько километров прошла группа в четвёртый день? Введи в поле ответа число без единиц измерения. При необходимости ввести десятичную дробь, разделяй её целую и дробную части запятой, без пробелов.'
sentence = 'Во дворе кот Геннадий охотился на напыщенного толстого голубя. (1) крадущегося кота была равна 1 м/с. Но голубь оказался не так прост и скоро, через (2), заметил приближающегося Геннадия. Птица взлетела на крышу гаража высотой в (3), чтобы отвязаться от кота. Но тот быстро преодолел (4) до этого строения в 5 метров, прыгнул наверх и почти ухватил пернатого за хвост! Голубь, конечно, не стерпел такой наглости и взлетел со скоростью (5). Обиженный Геннадий посмотрел некоторое (6) на улетающую птицу и спрыгнул вниз. Какова длина траектории (в метрах) кота Геннадия за описанную утреннюю прогулку?'
#sentence = 'У Миши было 3 три мячика. Два из них он отдал Даше. Сколько мячиков осталось у Миши?'
category_from = 0
category_to = 2

In [ ]:
print(sentence)
sentence_with_masks, masks, prev_words = MaskCreator().mask(sentence, category_from)
print(sentence_with_masks, masks)
new_masks = ['' for i in range(len(masks))]
words_in_sentence = word_tokenize(sentence_with_masks)
get_first_mask = False
now_ind_to_prev_ind = {}
count_masks = 0
for ind, word in enumerate(words_in_sentence):
  if word[:4] == 'mask':
    now_ind_to_prev_ind[ind] = count_masks
    count_masks += 1
for word in words_in_sentence:
  if word[:4] == 'mask' and masks[int(word[4:])] in nouns_actors[category_from]:
    first_mask = navec[masks[int(word[4:])]]
    new_word = nouns_actors[category_to][random.randint(0, 3)]
    new_first_mask = navec[new_word]
    new_masks[int(word[4:])] = new_word
    get_first_mask = True
for ind, word in enumerate(words_in_sentence):
  if word[:4] == 'mask':
    if not get_first_mask:
      first_mask = navec[masks[int(word[4:])]]
      new_word = nouns[category_to][random.randint(0, 19)]
      new_first_mask = navec[new_word]
      new_masks[int(word[4:])] = new_word
      get_first_mask = True
    elif new_masks[int(word[4:])] == '':
      min_val = 1
      min_new_word = ''
      #ideal_dist = first_mask - navec[masks[int(word[4:])]]
      ideal_dist = navec[masks[int(word[4:])]] - navec[categories[category_from]] + navec[categories[category_to]]
      for new_word in nouns[category_to]:
        if not new_word in navec or new_word in new_masks:
          continue
        now_dist = cos_dist.cosine(navec[new_word], new_first_mask - ideal_dist)
        if now_dist < min_val:
          min_val = now_dist
          min_new_word = new_word
      if min_val > 0.5:
        min_new_word = masks[int(word[4:])]
      new_masks[int(word[4:])] = min_new_word
    put_word = new_masks[int(word[4:])]
    prev_word = prev_words[now_ind_to_prev_ind[ind]]
    case_prev_word = morph.parse(prev_word)[0].tag.case
    number_prev_word = morph.parse(prev_word)[0].tag.number
    words_in_sentence[ind] = morph.parse(put_word)[0].inflect({number_prev_word, case_prev_word})[0]

sentence = TreebankWordDetokenizer().detokenize(words_in_sentence)

print(sentence)

Во дворе кот Геннадий охотился на напыщенного толстого голубя. (1) крадущегося кота была равна 1 м/с. Но голубь оказался не так прост и скоро, через (2), заметил приближающегося Геннадия. Птица взлетела на крышу гаража высотой в (3), чтобы отвязаться от кота. Но тот быстро преодолел (4) до этого строения в 5 метров, прыгнул наверх и почти ухватил пернатого за хвост! Голубь, конечно, не стерпел такой наглости и взлетел со скоростью (5). Обиженный Геннадий посмотрел некоторое (6) на улетающую птицу и спрыгнул вниз. Какова длина траектории (в метрах) кота Геннадия за описанную утреннюю прогулку?
во mask0 mask1 геннадий охотился на напыщенного толстого голубя . (1) крадущегося mask1 была равна 1 м / с . но голубь оказался не так прост и скоро, через (2 ), заметил приближающегося геннадия . mask2 взлетела на крышу гаража высотой в (3 ), чтобы отвязаться от mask1 . но тот быстро преодолел (4) до этого строения в 5 метров, прыгнул наверх и почти ухватил пернатого за mask3! голубь, конечно, не

In [ ]:
cos_dist.cosine(navec['птица'], navec['игрок'] + navec['животные'] - navec['спорт'])

0.7580268085002899

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForMaskedLM.from_pretrained("bert-base-multilingual-cased")

In [ ]:
words_in_sentence = word_tokenize(sentence)
to_word = {}
count_masks = 0
for ind, word in enumerate(words_in_sentence):
  if morph.parse(word)[0].tag.POS == 'INFN' or morph.parse(word)[0].tag.POS == 'VERB':
    words_in_sentence[ind] = '[MASK]'
    count_masks += 1
    continue
  '''
  normal_word = morph.parse(word)[0].normal_form
  case_word = morph.parse(word)[0].tag.case
  number_word = morph.parse(word)[0].tag.number
  if normal_word in nouns_actors[category_from]:
    if to_word.get(normal_word) == None:
      to_word[normal_word] = nouns_actors[category_to][random.randint(0, 19)]
    # Заменяю либо на существительное из другой категории, либо на маску (тестировал разные варианты)
    #words_in_sentence[ind] = morph.parse(to_word[normal_word])[0].inflect({number_word, case_word})[0]
    words_in_sentence[ind] = '[MASK]'
    count_masks += 1
  if normal_word in nouns[category_from]:
    if to_word.get(normal_word) == None:
      to_word[normal_word] = nouns[category_to][random.randint(0, 19)]
    #words_in_sentence[ind] = morph.parse(to_word[normal_word])[0].inflect({number_word, case_word})[0]
    words_in_sentence[ind] = '[MASK]'
    count_masks += 1'''

In [ ]:
sentence = TreebankWordDetokenizer().detokenize(words_in_sentence)
sentence = sentence.replace(' .', '.')

sentence

'1. 4. Во время летних соревнование по океан папа [MASK] пруд. длина плавательной полоса в пруд 25 м. После того как первый художник соревнование [MASK] часть полоса, ему [MASK] [MASK] 10 м. Сколько метр уже [MASK] художник соревнование?'

In [ ]:
tokenized = tokenizer(sentence, return_tensors="pt")
with torch.no_grad():
  outputs = model(**tokenized)
masked_index = np.where(torch.flatten(tokenized['input_ids']).numpy() == 103)
for i in masked_index[0]:
  predicted_index = torch.argmax(outputs[0][0][i]).item()
  predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
  ind = sentence.split().index('[MASK]')
  sentence = sentence.split()
  sentence[ind] = predicted_token
  sentence = ' '.join(sentence)

In [ ]:
sentence

'1. 4. Во время летних соревнование по океан папа по пруд. длина плавательной полоса в пруд 25 м. После того как первый художник соревнование . часть полоса, ему длина ##о 10 м. Сколько метр уже первый художник соревнование?'